In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import requests
import random
import time
from selenium.webdriver import Firefox
from scrape_functions import (
    get_movie_scores,
    get_all_titles
)

In [68]:
browser = Firefox()
url = "https://www.rottentomatoes.com/browse/in-theaters?minTomato=0&maxTomato=100&minPopcorn=0&maxPopcorn=100&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=popularity"
browser.get(url)

In [69]:
show_more = browser.find_element_by_css_selector(".btn-secondary-rt")

In [70]:
def show_more_click(browser):
    if show_more.is_displayed: 
        show_more.click()
    else:
        pass
    return None

In [71]:
show_more_click(browser)

In [72]:
movie_scores_list = get_movie_scores(browser)

In [73]:
movie_scores_list[0].text

' 31% 50%'

In [74]:
scores=[]
for movie in movie_scores_list:
    one_score = movie.text
    scores.append(one_score)

In [134]:
scores_lean = [score.replace(' ','') for score in scores]

In [77]:
scores_split = [score_lean.split('%') for score_lean in scores_lean]

split_scores = [score_split.pop() for score_split in scores_split]

In [79]:
critics_score = [score[0] for score in scores_split]
audience_score = [score[1] for score in scores_split]

In [80]:
all_titles = get_all_titles(browser)

In [82]:
titles=[]
for movie in all_titles:
    one_title = movie.text
    titles.append(one_title)

In [84]:
import pandas as pd

In [85]:
dictionary = {'title':titles, 'critics_score':critics_score, 'audience_score':audience_score}

In [99]:
rt_df = pd.DataFrame(dictionary)

In [135]:
rt_df = rt_df.drop('_id', axis=1)

In [87]:
import pymongo

In [88]:
mc = pymongo.MongoClient()

In [89]:
db = mc['movie']

In [90]:
rt_coll = db['rotten_tomatoes']

In [91]:
rt_coll.insert_one(dictionary)

In [92]:
from fan_scrape_functions import (
    get_theater_title
    ,get_movie_titles
    ,get_theater_ids)

In [93]:
browser = Firefox()
url = "https://www.fandango.com/seattle_wa_movietimes"
browser.get(url)

In [94]:
theaters = get_theater_ids(browser)

dict_movies = get_movie_titles(browser,theaters)


In [97]:
theater_listings = pd.DataFrame(dict_movies)

In [143]:
new_titles1 = theater_listings['title'].apply(lambda x: x.replace(' (2019)',''))   

new_titles2 = new_titles1.apply(lambda x: x.replace(' 3D',''))   
new_titles = new_titles2.apply(lambda x: x.title())
theater_listings['title'] = new_titles


In [140]:
final = pd.merge(theater_listings,
                 rt_df,
                 left_on='title',
                 right_on='title', how='left'
                )


In [153]:
final['audience_score'] = final['audience_score'].astype(int, errors='ignore')

In [154]:
final['critics_score'] = final['critics_score'].astype(int, errors='ignore')

In [156]:
final.sort_values('audience_score', ascending=False,)

,theater,times,title,critics_score,audience_score
4,REGAL MERIDIAN & 4DX,[11:00p],Shazam!,90,87
11,AMC PACIFIC PLACE 11,"[11:00a, 1:45p, 5:10p, 7:05p, 10:35p]",Shazam!,90,87
21,SIFF CINEMA UPTOWN,"[4:00p, 6:30p, 9:00p]",Rafiki,91,83
1,REGAL MERIDIAN & 4DX,"[4:15p, 6:40p, 11:20p]",Breakthrough,65,78
8,AMC PACIFIC PLACE 11,"[10:50a, 2:50p, 4:05p, 6:45p]",Breakthrough,65,78
10,AMC PACIFIC PLACE 11,"[10:55a, 2:05p, 5:20p]",Little,46,74
5,REGAL MERIDIAN & 4DX,"[3:30p, 6:45p]",Us,94,66
13,AMC PACIFIC PLACE 11,"[10:25a, 1:10p, 4:00p, 7:45p]",Us,94,66
6,REGAL MERIDIAN & 4DX,[10:50p],Captain Marvel,78,56
12,AMC PACIFIC PLACE 11,"[10:05a, 1:05p, 4:10p, 7:25p, 10:30p]",Dumbo,47,55
